### Scrape Data form omdb API

In [2]:
# import pandas as pd
# import requests
# import time

# def fetch_movie_data(api_key, movie_ids):
#     base_url = 'http://www.omdbapi.com/'
#     movie_data = []

#     for movie_id in movie_ids:
#         response = requests.get(base_url, params={
#             'apikey': api_key,
#             'i': movie_id,
#             'r': 'json',
#             'plot': 'full'  # Fetch full plot details
#         })
#         data = response.json()
#         if data.get('Response') == 'True':
#             movie_data.append(data)
#         else:
#             print(f"Error fetching data for movie ID {movie_id}: {data.get('Error')}")
#         time.sleep(2)  # To respect API rate limits

#     return movie_data

# def read_csv(file_path):
#     df = pd.read_csv(file_path)
#     return df['movie_id'].tolist()

# def save_to_csv(data, file_path):
#     df = pd.DataFrame(data)
#     df.to_csv(file_path, index=False)

# # api_key = '6463f6d2'
# api_key = os.getenv("OMDB_API_KEY")
# input_csv = 'hindi_data/IMDB-Movie-Dataset(2023-1951).csv'
# output_csv = 'hindi_data/scraped_hindi_movie_data.csv'

# movie_ids = read_csv(input_csv)

# movie_data = fetch_movie_data(api_key, movie_ids)

# save_to_csv(movie_data, output_csv)

# print(f"Data for {len(movie_data)} movies saved to {output_csv}")


### RAG - Movie Recomm

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Langchain
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.chains.query_constructor.base import AttributeInfo

from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from tqdm.autonotebook import tqdm

c:\Users\auth\OneDrive\Desktop\movieRecomm\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Pinecone

In [4]:
loader = DirectoryLoader(
    path="./hindi_data/final data",
    loader_cls=CSVLoader,
    show_progress=True)

docs = loader.load()


100%|██████████| 1/1 [00:00<00:00, 11.90it/s]


In [5]:

metadata_field_info = [
    AttributeInfo(name="Title", description="The title of the movie", type="string"),
    AttributeInfo(name="Year", description="The year the movie was released", type="integer"),
    AttributeInfo(name="Released", description="The release date of the movie", type="string"),
    AttributeInfo(name="Runtime", description="The runtime of the movie in minutes", type="string"),
    AttributeInfo(name="Genre", description="The genre of the movie", type="string"),
    AttributeInfo(name="Director", description="The director of the movie", type="string"),
    AttributeInfo(name="Actors", description="The actors in the movie", type="string"),
    AttributeInfo(name="Plot", description="The plot of the movie", type="string"),
    AttributeInfo(name="Poster", description="The URL of the movie poster", type="string"),
    AttributeInfo(name="Ratings", description="The ratings of the movie", type="string"),
    AttributeInfo(name="imdbRating", description="The IMDb rating of the movie", type="string"),
    AttributeInfo(name="imdbID", description="The IMDb ID of the movie", type="string"),
    AttributeInfo(name="Website", description="The official website", type="string")
]

In [6]:
fields_to_convert_list = ["Genre", "Actors", "Ratings"]
fields_to_convert_int = ["Year", "imdbRating"]

In [7]:
def convert_to_list(doc, field):
    if field in doc.metadata and doc.metadata[field]:
        doc.metadata[field] = doc.metadata[field].split(", ")

def convert_to_int(doc, field):
    if field in doc.metadata and doc.metadata[field]:
        try:
            doc.metadata[field] = int(doc.metadata[field])
        except ValueError:
            pass 

In [8]:
for doc in docs:

    page_content_dict = dict(line.split(": ", 1)
                            for line in doc.page_content.split("\n") if ": " in line)
    

    doc.page_content = 'Overview: ' + page_content_dict.get('Plot', '') + '. Keywords: ' + page_content_dict.get('Genre', '')
    

    doc.metadata = {field.name: page_content_dict.get(field.name) for field in metadata_field_info}
    
    for field in fields_to_convert_list:
        convert_to_list(doc, field)
    
    for field in fields_to_convert_int:
        convert_to_int(doc, field)

In [9]:
len(docs)

979

In [10]:
docs[111]

Document(page_content="Overview: When Raj (Shah Rukh Khan) and Simran (Kajol) first met on an inter-rail holiday in Europe, it wasn't exactly Love at first sight but when Simran is taken back to India for an arranged marriage, things change. Encouraged by his father Dharamvir (Anupam Kher), Raj decides to fly down from London to not just win his Bride but her whole family and the blessings of her father Baldev Singh (Amrish Puri).. Keywords: Drama, Romance", metadata={'Title': 'Dilwale Dulhania Le Jayenge', 'Year': 1995, 'Released': '20 Oct 1995', 'Runtime': '181 min', 'Genre': ['Drama', 'Romance'], 'Director': 'Aditya Chopra', 'Actors': ['Shah Rukh Khan', 'Kajol', 'Amrish Puri'], 'Plot': "When Raj (Shah Rukh Khan) and Simran (Kajol) first met on an inter-rail holiday in Europe, it wasn't exactly Love at first sight but when Simran is taken back to India for an arranged marriage, things change. Encouraged by his father Dharamvir (Anupam Kher), Raj decides to fly down from London to not

In [11]:
docs[111].page_content

"Overview: When Raj (Shah Rukh Khan) and Simran (Kajol) first met on an inter-rail holiday in Europe, it wasn't exactly Love at first sight but when Simran is taken back to India for an arranged marriage, things change. Encouraged by his father Dharamvir (Anupam Kher), Raj decides to fly down from London to not just win his Bride but her whole family and the blessings of her father Baldev Singh (Amrish Puri).. Keywords: Drama, Romance"

In [12]:
docs[111].metadata

{'Title': 'Dilwale Dulhania Le Jayenge',
 'Year': 1995,
 'Released': '20 Oct 1995',
 'Runtime': '181 min',
 'Genre': ['Drama', 'Romance'],
 'Director': 'Aditya Chopra',
 'Actors': ['Shah Rukh Khan', 'Kajol', 'Amrish Puri'],
 'Plot': "When Raj (Shah Rukh Khan) and Simran (Kajol) first met on an inter-rail holiday in Europe, it wasn't exactly Love at first sight but when Simran is taken back to India for an arranged marriage, things change. Encouraged by his father Dharamvir (Anupam Kher), Raj decides to fly down from London to not just win his Bride but her whole family and the blessings of her father Baldev Singh (Amrish Puri).",
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMDQ2ZmE2NTMtZDE3NC00YzFjLWJhNmEtMDEzMTI5ZjU2ZGM0XkEyXkFqcGdeQXVyNTkzNDQ4ODc@._V1_SX300.jpg',
 'Ratings': ["[{'Source': 'Internet Movie Database'",
  "'Value': '8.0/10'}",
  "{'Source': 'Rotten Tomatoes'",
  "'Value': '100%'}]"],
 'imdbRating': '8.0',
 'imdbID': 'tt0112870',
 'Website': 'N/A'}

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = text_splitter.split_documents(docs)

In [14]:
len(documents)

1118

In [15]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

In [16]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")

llm = Ollama(model="llama3",
            temperature=0,
            # streaming=True,
)

In [17]:
# import google.generativeai as genai
# from dotenv import load_dotenv
# import os
# from langchain_google_genai import ChatGoogleGenerativeAI

# load_dotenv()
# genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
# llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [18]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings

# embeddings=GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [19]:
temp = documents[0:31]

In [20]:
temp

[Document(page_content='Overview: A prison warden recruits inmates to commit outrageous crimes that shed light on corruption and injustice - and that lead him to an unexpected reunion.. Keywords: Action, Drama, Musical', metadata={'Title': 'Jawan', 'Year': 2023, 'Released': '07 Sep 2023', 'Runtime': '169 min', 'Genre': ['Action', 'Drama', 'Musical'], 'Director': 'Atlee', 'Actors': ['Shah Rukh Khan', 'Nayanthara', 'Vijay Sethupathi'], 'Plot': 'A prison warden recruits inmates to commit outrageous crimes that shed light on corruption and injustice - and that lead him to an unexpected reunion.', 'Poster': 'https://m.media-amazon.com/images/M/MV5BOWI5NmU3NTUtOTZiMS00YzA1LThlYTktNDJjYTU5NDFiMDUxXkEyXkFqcGdeQXVyMTUzNjEwNjM2._V1_SX300.jpg', 'Ratings': ["[{'Source': 'Internet Movie Database'", "'Value': '7.0/10'}]"], 'imdbRating': '7.0', 'imdbID': 'tt15354916', 'Website': 'N/A'}),
 Document(page_content='Overview: A single mother and her daughter commit a crime and a neighbor helps them cover 

In [21]:
index = PineconeVectorStore.from_documents(temp,embeddings,index_name="rag-movie-recomm")

In [22]:
retriever = index.as_retriever()

In [23]:
# retriever = index.as_retriever()

# def retrieve_query(query,k=2):
#     matching_results=index.similarity_search(query,k=k)
#     return matching_results

# # retrieve_query(query)

In [24]:
# from langchain.chains.question_answering import load_qa_chain

# chain=load_qa_chain(llm,chain_type="stuff")

In [25]:
# def retrieve_answers(query):
#     doc_search=retrieve_query(query)
#     print(doc_search)
#     response=chain.run(input_documents=doc_search,question=query)
#     return response

In [26]:
# our_query = "Recommend me 3 movies on Keywords Romance/Love"
# answer = retrieve_answers(our_query)
# print(answer)

In [27]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            """
            Recommend films based on the user's query and the provided context only. 
            Only recommend films if they are in the context and relevant.  
            Provide three to five relevant film recommendations, ensuring from the context only. 
            If the context is empty or none of the films are relevant, dont give anything you can return empty json response.
            Do not recommend more than five films.
            Don't give answer on based of your knowledge, you are strictly to use context provided for recommending the movies only.
            
            your response should be in json format only strictly.
            
            Use Context provided to give output of recommended movies in the below mentioned format for every movie, In the reasoning field in the output format, write your reason for recommending the movie make it as you are explaining why its a must watch movie:-
            
            OUTPUT FORMAT:
            [
                "Title": "",
                "Plot": "",
                "Genre":""
                "Runtime": "",
                "Release Year": "",
                "Actor": "",
                "Director": "",
                "Website": "",
                "Poster": "",
                "IMDB Rating": "",
                "imdbID": "",
                "Reasoning": ""
            ]
            
            Question: {question}
            Context: {context}
            """
        ),
    ]
)

In [28]:
def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        metadata = doc.metadata
        formatted_doc = {
            "Title": metadata.get('Title', 'N/A'),
            "Plot": metadata.get('Plot', 'N/A'),
            "Genre": metadata.get('Genre', 'N/A'),
            "Runtime": metadata.get('Runtime', 'N/A'),
            "Release Year": metadata.get('Year', 'N/A'),
            "Actor": ', '.join(metadata.get('Actors', [])),
            "Director": metadata.get('Director', 'N/A'),
            "Website": metadata.get('Website', 'N/A'),
            "Poster": metadata.get('Poster', 'N/A'),
            "IMDB Rating": metadata.get('imdbRating', 'N/A'),
            "imdbID": metadata.get('imdbID', 'N/A'),
            "Reasoning": ""
        }
        formatted_docs.append(formatted_doc)
    return formatted_docs

In [29]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
)

response = rag_chain.invoke({
    "question": "Give me recommendations about crime"
})

print(response)

[
    {
        "Title": "Neeyat",
        "Plot": "When a mysterious murder takes place at exiled billionaire Ashish Kapoor's birthday getaway, detective Mira Rao must use all her skill to unravel devious motives as the suspects are Kapoor's close family and friends.",
        "Runtime": "134 min",
        "Release Year": 2023.0,
        "Actor": "Vidya Balan, Ram Kapoor, Rahul Bose",
        "Director": "Anu Menon",
        "Website": "N/A",
        "Poster": "https://m.media-amazon.com/images/M/MV5BZDMwYzM5NWItNDU1ZS00MDljLWIxZmQtNDI5MzBlZDJlODBlXkEyXkFqcGdeQXVyNTM1MzM4Nzk@._V1_SX300.jpg",
        "IMDB Rating": "5.3",
        "imdbID": "tt21908676",
        "Reasoning": "This crime thriller has a complex plot and intriguing characters, making it a must-watch for fans of the genre."
    },
    {
        "Title": "Por Thozhil",
        "Plot": "A bright but faint-hearted rookie cop has to overcome his fears in order to succeed in his first case which sees him partnered with a reclusi

In [30]:
response

'[\n    {\n        "Title": "Neeyat",\n        "Plot": "When a mysterious murder takes place at exiled billionaire Ashish Kapoor\'s birthday getaway, detective Mira Rao must use all her skill to unravel devious motives as the suspects are Kapoor\'s close family and friends.",\n        "Runtime": "134 min",\n        "Release Year": 2023.0,\n        "Actor": "Vidya Balan, Ram Kapoor, Rahul Bose",\n        "Director": "Anu Menon",\n        "Website": "N/A",\n        "Poster": "https://m.media-amazon.com/images/M/MV5BZDMwYzM5NWItNDU1ZS00MDljLWIxZmQtNDI5MzBlZDJlODBlXkEyXkFqcGdeQXVyNTM1MzM4Nzk@._V1_SX300.jpg",\n        "IMDB Rating": "5.3",\n        "imdbID": "tt21908676",\n        "Reasoning": "This crime thriller has a complex plot and intriguing characters, making it a must-watch for fans of the genre."\n    },\n    {\n        "Title": "Por Thozhil",\n        "Plot": "A bright but faint-hearted rookie cop has to overcome his fears in order to succeed in his first case which sees him partn

In [31]:
import json

json_data = response[response.find("["):response.rfind("]") + 1]
output_data = json.loads(json_data)
print(output_data)


[{'Title': 'Neeyat', 'Plot': "When a mysterious murder takes place at exiled billionaire Ashish Kapoor's birthday getaway, detective Mira Rao must use all her skill to unravel devious motives as the suspects are Kapoor's close family and friends.", 'Runtime': '134 min', 'Release Year': 2023.0, 'Actor': 'Vidya Balan, Ram Kapoor, Rahul Bose', 'Director': 'Anu Menon', 'Website': 'N/A', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZDMwYzM5NWItNDU1ZS00MDljLWIxZmQtNDI5MzBlZDJlODBlXkEyXkFqcGdeQXVyNTM1MzM4Nzk@._V1_SX300.jpg', 'IMDB Rating': '5.3', 'imdbID': 'tt21908676', 'Reasoning': 'This crime thriller has a complex plot and intriguing characters, making it a must-watch for fans of the genre.'}, {'Title': 'Por Thozhil', 'Plot': 'A bright but faint-hearted rookie cop has to overcome his fears in order to succeed in his first case which sees him partnered with a reclusive senior officer to catch a serial killer on the loose.', 'Runtime': '147 min', 'Release Year': 2023.0, 'Actor': 'R. Sa

In [32]:
output_data[0]

{'Title': 'Neeyat',
 'Plot': "When a mysterious murder takes place at exiled billionaire Ashish Kapoor's birthday getaway, detective Mira Rao must use all her skill to unravel devious motives as the suspects are Kapoor's close family and friends.",
 'Runtime': '134 min',
 'Release Year': 2023.0,
 'Actor': 'Vidya Balan, Ram Kapoor, Rahul Bose',
 'Director': 'Anu Menon',
 'Website': 'N/A',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BZDMwYzM5NWItNDU1ZS00MDljLWIxZmQtNDI5MzBlZDJlODBlXkEyXkFqcGdeQXVyNTM1MzM4Nzk@._V1_SX300.jpg',
 'IMDB Rating': '5.3',
 'imdbID': 'tt21908676',
 'Reasoning': 'This crime thriller has a complex plot and intriguing characters, making it a must-watch for fans of the genre.'}

In [33]:
output_data[1]

{'Title': 'Por Thozhil',
 'Plot': 'A bright but faint-hearted rookie cop has to overcome his fears in order to succeed in his first case which sees him partnered with a reclusive senior officer to catch a serial killer on the loose.',
 'Runtime': '147 min',
 'Release Year': 2023.0,
 'Actor': 'R. Sarathkumar, Lisha Chinnu, Ashok Selvan',
 'Director': 'Vignesh Raja, T. Senthil Paramasivam',
 'Website': 'N/A',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BY2U5ZjQ2MGMtNDI5ZS00NzJjLTg5YTAtMTVhYWQ4MGE4NmQxXkEyXkFqcGdeQXVyMTY1MzAyNjU4._V1_SX300.jpg',
 'IMDB Rating': '8.0',
 'imdbID': 'tt27542289',
 'Reasoning': 'This crime drama has a strong focus on character development and features a gripping storyline, making it a must-watch for fans of the genre.'}

### Pinecone Retrival

In [176]:
embeddings = OllamaEmbeddings(model="llama3")
index_name="rag-movie-recomm"

In [177]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)
docs = docsearch.similarity_search("Give me movie recommendations about Love")

In [178]:
docs

[Document(page_content="Overview: A retired jailer goes on a manhunt to find his son's killers. But the road leads him to a familiar, albeit a bit darker place. Can he emerge from this complex situation successfully?. Keywords: Action, Comedy, Crime", metadata={'Actors': ['Rajinikanth', 'Mohanlal', 'Shivarajkumar'], 'Director': 'Nelson Dilipkumar', 'Genre': ['Action', 'Comedy', 'Crime'], 'Plot': "A retired jailer goes on a manhunt to find his son's killers. But the road leads him to a familiar, albeit a bit darker place. Can he emerge from this complex situation successfully?", 'Poster': 'https://m.media-amazon.com/images/M/MV5BNzFkNmZkMTctYjRhMS00ODZiLWFlNjQtZmZmN2IzMDJlNmVkXkEyXkFqcGdeQXVyMTY1MzAyNjU4._V1_SX300.jpg', 'Ratings': ["[{'Source': 'Internet Movie Database'", "'Value': '7.1/10'}]"], 'Released': '10 Aug 2023', 'Runtime': '168 min', 'Title': 'Jailer', 'Website': 'N/A', 'Year': 2023.0, 'imdbID': 'tt11663228', 'imdbRating': '7.1'}),
 Document(page_content='Overview: Much to the

### Ollama api call

In [30]:
# import requests
# import base64
# import json

# def generate_response(model, prompt):
#     url = 'http://localhost:11434/api/generate'  
#     data = {
#         'model': model,
#         'prompt': prompt
#     }

#     response = requests.post(url, json=data)
#     response_content = response.content.decode("utf-8") 
#     json_objects = [json.loads(obj) for obj in response_content.strip().split("\n")]
#     print(json_objects)
#     responses = [item.get('response', '') for item in json_objects]
#     combined_response = ' '.join(responses)
#     sentences = combined_response.split(". ")
#     formatted_response = "".join([" " + sentence + "." for sentence in sentences])
#     print(formatted_response)
#     print('\n')

# model_name = "llama3"
# query = "Recommend me movies about god/goddess, horror ones" 

# generate_response(model_name, query)

[{'model': 'llama3', 'created_at': '2024-05-31T07:12:20.62375562Z', 'response': 'What', 'done': False}, {'model': 'llama3', 'created_at': '2024-05-31T07:12:20.916239115Z', 'response': ' a', 'done': False}, {'model': 'llama3', 'created_at': '2024-05-31T07:12:21.164111711Z', 'response': ' fascinating', 'done': False}, {'model': 'llama3', 'created_at': '2024-05-31T07:12:21.488562706Z', 'response': ' combination', 'done': False}, {'model': 'llama3', 'created_at': '2024-05-31T07:12:21.739449402Z', 'response': '!', 'done': False}, {'model': 'llama3', 'created_at': '2024-05-31T07:12:22.005689498Z', 'response': ' Here', 'done': False}, {'model': 'llama3', 'created_at': '2024-05-31T07:12:22.272500493Z', 'response': ' are', 'done': False}, {'model': 'llama3', 'created_at': '2024-05-31T07:12:22.642049788Z', 'response': ' some', 'done': False}, {'model': 'llama3', 'created_at': '2024-05-31T07:12:23.075108381Z', 'response': ' movie', 'done': False}, {'model': 'llama3', 'created_at': '2024-05-31T07:

### Gemini api call


In [ ]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI

class GeneralQuestionAnswering:
    def __init__(self):
        # Load environment variables
        load_dotenv()

        # Configure the Gemini model with the API key
        genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
        # self.model = genai.model("gemini-pro")
        self.model = ChatGoogleGenerativeAI(model="gemini-pro")

    def ask_question(self, question):
        # Define the prompt template
        prompt_template = """
            You are an expert in answering questions.
            You are tasked to provide a detailed and accurate answer to the given question:

            NOTE: Use only the information you have to provide the answer.

            Provide the output response in plain text.

            I will tip you $1000 if the user finds the answer helpful.

            <question>
            {user_question}
            </question>
        """
        
        # Format the prompt with the user's question
        prompt = prompt_template.format(user_question=question)

        # Generate a response using the Gemini model
        response = self.model.invoke(prompt)
        # print(response)

        return response.content

if __name__ == "__main__":
    qa = GeneralQuestionAnswering()
    question = input("Enter your question: ")
    answer = qa.ask_question(question)
    print("Answer:", answer)


In [2]:
# !recommend recommend me movies on action
# !recommend recommend me movies on drama
# !recommend suggest best movie to watch with family
# !recommend give me 3 movies latest hits
